In [1]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
debug = 2
import numpy as np
import pandas
import time
import geopandas
import os
import re
from shutil import copyfile
pandas.set_option('display.max_colwidth', -1)

output_basedir = '/home/idies/workspace/Temporary/raddick/cra_scratch_final/'
acs5_dir = output_basedir + 'acs5/'
outdir = acs5_dir

census_basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'

extrasdir = '/home/idies/workspace/Storage/raddick/census/extras/'
baltimore_dir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/baltimore/'

g = 0
print('ok')

ok


In [2]:
# IV: population by race; owner-occupied units; MFI; hs grad pct (age 25 and older); 
### female hoh pct; unemployment pct (age 18 and older); poverty pct; median home value;
### median home year built
s = time.time()

varlist = ['STUSAB','LOGRECNO']    # Lookup variables (we always need these): Seq 1 in 2017
varlist += ['B01001_001']   # Total population: Seq 2 in 2017
varlist += ['B11001_001']  # Total households: Seq 36 in 2017
varlist += ['B02001_002', 'B02001_003'] # White-only and black-only population: Seq 4 in 2017

# WHITE-ONLY AND BLACK-ONLY HOUSEHOLDER
varlist += ['B11001A_001', 'B11001B_001']

varlist += ['B25003_002'] # Owner-occupied housing units: Seq 103 in 2017
varlist += ['B19113_001'] # Median family income: Seq 64 in 2017

# EDUCATION HIGH SCHOOL OR ABOVE
varlist += ['B15002_011', 'B15002_012', 'B15002_013', 'B15002_014', 'B15002_015']
varlist += ['B15002_016', 'B15002_017', 'B15002_018']
varlist += ['B15002_028', 'B15002_029', 'B15002_030', 'B15002_031', 'B15002_032']
varlist += ['B15002_033', 'B15002_034', 'B15002_035']

varlist += ['B11001_006'] # Female householder no husband present: Seq 36 in 2017

varlist += ['B23025_005'] # Unemployed people age 16 or over: Seq 79 in 2017
varlist += ['B17001_002'] # Income in the past 12 months below poverty level: Seq 48 in 2017
varlist += ['B25077_001'] # MEDIAN VALUE (DOLLARS)% Owner-occupied housing units%Median value (dollars): Seq 106 in 2017
varlist += ['B25035_001'] # MEDIAN YEAR STRUCTURE BUILT% Housing units%Median year structure built: Seq 105 in 2017

possible_ind_vars = varlist
# ANOTHER OPTION: UNEMPLOYED PEOPLE AGES 22 AND OLDER: Seq 76 in 2017
#varlist += metadata_df['variable'][
#    (metadata_df['description'].apply(lambda x: 'unemplo' in x.lower()))
#    & (metadata_df['description'].apply(lambda x: x[0:31] == 'SEX BY AGE BY EMPLOYMENT STATUS'))
#    & (metadata_df['variable'].apply(lambda x: re.search("[A-Z]_", x) == None))  # note: case-sensitive search for varnames
#    #& (metadata_df['sequence_number'].apply(lambda x: ((x >= 76) & (x <= 76))))    
#].tolist()

# Cross-tab variables to come back to later
#varlist += ['B11001A_006', 'B11001B_006', 'B17001A_002', 'B17001B_002']
#varlist += ['C15002A_004', 'C15002A_009', 'C15002B_004', 'C15002B_009']
#varlist += ['B19113A_001', 'B19113B_001', 'B25003A_002', 'B25003B_002']

# CHECK!
#metadata_df[metadata_df['variable'].apply(lambda x: x in varlist)]
### need to calculate percentages: population 25 and older...
##### population age 16 and older in labor force, population for whom poverty status determined
comparison_vars = []
# POPULATION AGE 25 OR OVER
comparison_vars += ['B01001_011', 'B01001_012', 'B01001_013', 'B01001_014', 'B01001_015']
comparison_vars += ['B01001_016', 'B01001_017', 'B01001_018', 'B01001_019', 'B01001_020']
comparison_vars += ['B01001_021', 'B01001_022', 'B01001_023', 'B01001_024', 'B01001_025']
comparison_vars += ['B01001_035', 'B01001_036', 'B01001_037', 'B01001_038', 'B01001_039']
comparison_vars += ['B01001_040', 'B01001_041', 'B01001_042', 'B01001_043', 'B01001_044']
comparison_vars += ['B01001_045', 'B01001_046', 'B01001_047', 'B01001_048', 'B01001_049']

comparison_vars += ['B11001_002', 'B11001_007'] # Family and nonfamily households

#if (thisyear != 2010):
comparison_vars += ['B23025_002'] # EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER%Population 16 years and over%Total%In labor force: Seq 79 in 2017
comparison_vars += ['B17001_001'] # POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY AGE% Population for whom poverty status is determined%Total: Seq 48 in 2017

# CHECK!
#metadata_df[metadata_df['variable'].apply(lambda x: x in comparison_vars)]

if (len(varlist) < len(possible_ind_vars) + len(comparison_vars)):
    varlist += comparison_vars

# CHECK!
#metadata_df[metadata_df['variable'].apply(lambda x: x in varlist)]
e = time.time()
g = g + (e-s)

print('Defined variables we need in {0:,.2f} seconds!'.format(e-s))
#for x in varlist:
#    print(metadata_df['variable'][metadata_df['variable'] == x].values[0])#, ',"',
#          metadata_df['description'][metadata_df['variable'] == x].values[0],'"')


Defined variables we need in 0.00 seconds!


In [5]:
g = 0
state_codes_df = pandas.read_csv(extrasdir+'statecodes.csv')
state_codes_df = state_codes_df.set_index('STUSAB')
states = state_codes_df.index.values.tolist()
states = sorted([x.lower() for x in states if x not in ('MD', 'AS', 'GU', 'MP', 'PR', 'UM', 'VI')])
#states = states[-7:]
#states = ['md']

#years = np.arange(2017, 2009, -1)
years = np.arange(2010, 2009, -1)

for thisyear in years:
    s = time.time()
    print('Parsing {0:.0f}...'.format(thisyear))    
    metaoutfile = outdir + 'acs5_metadata_{0:.0f}.csv'.format(thisyear)
    census_yeardir = census_basedir + str(thisyear) + '/'
    census_geodir = census_yeardir + 'geography/'
    
    if (thisyear == 2017):
        census_metadir = census_yeardir + 'metadata/xls_temp/'
    elif (thisyear == 2016):
        census_metadir = census_yeardir + 'metadata/templates/'
    elif (thisyear == 2014):
        census_metadir = census_yeardir + 'metadata/seq/'
    else:
        census_metadir = census_yeardir + 'metadata/'
    if (thisyear == 2017):
        census_rawdatadir = census_yeardir + 'rawdata/'
    elif (thisyear == 2016):
        census_rawdatadir = census_yeardir + 'rawdata/data/tab4/sumfile/prod/{0:.0f}thru{1:.0f}/group2/'.format(thisyear-4, thisyear)
    elif (thisyear == 2014 or thisyear == 2012 or thisyear == 2010):
        census_rawdatadir = census_yeardir + 'rawdata/tab4/sumfile/prod/{0:.0f}thru{1:.0f}/group2/'.format(thisyear-4, thisyear)
    else:
        census_rawdatadir = census_yeardir + 'rawdata/group2/'.format(thisyear-4, thisyear)

    themetafiles = [x for x in os.listdir(census_metadir) if 'seq' in x.lower()]
    lastfilenum = len(themetafiles)
    metadata_df = pandas.DataFrame()

    i = 1
    if (thisyear == 2017):
        this_seq_metadata_cols_df = pandas.read_excel(census_metadir+'seq{0:.0f}.xlsx'.format(i))
    elif (thisyear == 2011):
        this_seq_metadata_cols_df = pandas.read_excel(census_metadir+'Seq{0:04d}.xls'.format(i))
    else:
        this_seq_metadata_cols_df = pandas.read_excel(census_metadir+'Seq{0:.0f}.xls'.format(i))
    this_seq_metadata_df = this_seq_metadata_cols_df.T
    this_seq_metadata_df.index.name = 'variable'
    this_seq_metadata_df = this_seq_metadata_df.rename(columns={0: 'description'})
    this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = i)
    this_seq_metadata_df = this_seq_metadata_df.reset_index()
    this_seq_metadata_df.index.name = 'varnum'
    #metadata_df = pandas.concat((metadata_df, this_seq_metadata_df))

    for i in range(1,lastfilenum+1):
        #if ((np.mod(i,50) == 0) | (i == lastfilenum)):
#            print('Parsing metadata for sequence {0:.0f}...'.format(i))
        if (thisyear == 2017):
            this_seq_metadata_cols_df = pandas.read_excel(census_metadir+'seq{0:.0f}.xlsx'.format(i))
        elif (thisyear == 2011):
            this_seq_metadata_cols_df = pandas.read_excel(census_metadir+'Seq{0:04d}.xls'.format(i))
        else:
            this_seq_metadata_cols_df = pandas.read_excel(census_metadir+'Seq{0:.0f}.xls'.format(i))
        this_seq_metadata_df = this_seq_metadata_cols_df.T
        this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = i)
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID', 'FILETYPE', 'STUSAB', 'CHARITER', 'SEQUENCE', 'LOGRECNO'], axis=0)
        this_seq_metadata_df.index.name = 'variable'
        this_seq_metadata_df = this_seq_metadata_df.rename(columns={0: 'description'})
        this_seq_metadata_df = this_seq_metadata_df.reset_index()
        metadata_df = pandas.concat((metadata_df, this_seq_metadata_df))
    metadata_df = metadata_df.reset_index(drop=True)
    metadata_df.index.name = 'varnum'
     
    #print('writing out metadata...')
    #metadata_df.to_csv(metaoutfile, encoding='utf-8')
    
    e = time.time()
    g = g + (e-s)
    print('Parsed {0:.0f} metadata in {1:,.0f} seconds!'.format(thisyear, e-s))

    #print('Reading {0:.0f} estimates and margins of error...'.format(thisyear))
    
    for onestate in states:
        s = time.time()
        outfile = outdir + 'acs5_tracts_for_cra_{0:.0f}{1:}.csv'.format(thisyear, onestate)
        
        #print('Parsing estimates and margins of error for {0:}...'.format(onestate))
        estimates_df = pandas.DataFrame()
        margins_of_error_df = pandas.DataFrame()
        for i in range(1, lastfilenum+1):
            #if ((i == 1) | (np.mod(i,25) == 0) | (i == lastfilenum)):
#                print('Processing sequence {0:,.0f}...'.format(i))
            this_seq_estimates_df = pandas.DataFrame()
            this_seq_margins_of_error_df = pandas.DataFrame()
            estimates_statefilename = census_rawdatadir + 'e{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i)        
            margins_of_error_statefilename = census_rawdatadir + 'm{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i)
            this_seq_estimates_df = pandas.read_csv(estimates_statefilename, header=None, sep=',', encoding='utf-8', low_memory=False)
            this_seq_margins_of_error_df = pandas.read_csv(margins_of_error_statefilename, header=None, sep=',', encoding='utf-8', low_memory=False)            
            column_names = metadata_df[metadata_df['sequence_number'] == i].set_index('variable').T.columns.tolist()
            column_names = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO'] + column_names

            this_seq_estimates_df.columns = column_names
            this_seq_margins_of_error_df.columns = column_names
            if (i == 1):
                estimates_df = this_seq_estimates_df
                margins_of_error_df = this_seq_margins_of_error_df
            else:
                this_seq_estimates_df = this_seq_estimates_df.drop(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], axis=1)
                this_seq_margins_of_error_df = this_seq_margins_of_error_df.drop(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], axis=1)
                estimates_df = estimates_df.merge(this_seq_estimates_df, how='left', on=['STUSAB','LOGRECNO'], sort=False)
                margins_of_error_df = margins_of_error_df.merge(this_seq_margins_of_error_df, how='left', on=['STUSAB','LOGRECNO'], sort=False)
        
        # e = time.time()
        #print('Parsed {0:.0f} data for {1:} in {2:.0f} seconds...'.format(thisyear, onestate, e-s))
        # s = time.time()
        #print('Getting only the variables we need...')
        if (thisyear == 2010):
            estimates_df = estimates_df.assign(B23025_002 = np.nan)
            estimates_df = estimates_df.assign(B23025_005 = np.nan)
            margins_of_error_df = margins_of_error_df.assign(B23025_002 = np.nan)
            margins_of_error_df = margins_of_error_df.assign(B23025_005 = np.nan)
        #print('Originally {0:,.0f} variables in estimates and {1:,.0f} in margins of error...'.format(estimates_df.shape[1], margins_of_error_df.shape[1]))
        estimates_df = estimates_df[varlist]
        margins_of_error_df = margins_of_error_df[varlist]
        #print('Now {0:,.0f} variables in estimates and {1:,.0f} in margins of error...'.format(estimates_df.shape[1], margins_of_error_df.shape[1]))
        
        estimates_df_bk = estimates_df
        margins_of_error_df_bk = margins_of_error_df
        #print('Joining estimates and margins of error...')
        for x in margins_of_error_df.columns[2:]:
            margins_of_error_df = margins_of_error_df.rename(columns={x: x+'_err'})
        acs5_df = estimates_df.merge(margins_of_error_df, how='left', on=['STUSAB', 'LOGRECNO'])

        #margins_of_error_df = margins_of_error_df_bk
        #error_columns = ['LOGRECNO'] + [x+'_err' for x in margins_of_error_df.columns[6:]]

        #print('Joining geography...')
        if (thisyear >= 2016):
            filename = census_geodir+'{0:}.xlsx'.format(onestate)
        else:
            filename = census_geodir+'{0:}.xls'.format(onestate)
        geo_df = pandas.read_excel(filename)
        if (thisyear == 2017):
            geo_df = geo_df.assign(STUSAB = geo_df['State'].apply(lambda x: x.lower()))
        else:
            geo_df = geo_df.assign(STUSAB = geo_df['STATE'].apply(lambda x: x.lower()))
        geo_df = geo_df.rename(columns={'Geography ID': 'GEOID'})
#if (debug >= 1):
#    print('Retaining only tract-level geographies...')
##    print('Retaining only tract- and block-group-level geographies...')
#geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140'))]
##geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140')) | (geo_df['GEOID'].apply(lambda x: x[0:3] == '150'))]

##geo_df = geo_df.set_index('GEOID')  #We'll set GEOID as index colum AFTER the merge
        if (thisyear == 2017):
            acs5_df = acs5_df.merge(geo_df, how='inner', left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
        else:
            acs5_df = acs5_df.merge(geo_df, how='inner', left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'LOGRECNO'])
        acs5_df = acs5_df.set_index('GEOID')

        #print('Fixing column names for easy joining later...')
        if (thisyear == 2017):
            acs5_df = acs5_df.drop('Logical Record Number', axis=1)
        if (acs5_df.columns[-2] != 'STATE'):
            acs5_df = acs5_df.rename(columns={acs5_df.columns[-2]: 'STATE'})
        if (acs5_df.columns[-1] != 'Geography Name'):
            acs5_df = acs5_df.rename(columns={acs5_df.columns[-1]: 'Geography Name'})

        #print('Writing out data...')
        acs5_df.to_csv(outfile, encoding='utf-8')
        metadata_df[metadata_df['variable'].apply(lambda x: x in varlist)].to_csv(metaoutfile, encoding='utf-8')

        #print('\nData written out to:\n{0:}\nMetadata written out to:\n{1:}'.format(outfile, metaoutfile))

        e = time.time()
        g = g + (e-s)

        print('{0:}: Wrote {1:,.0f} geographies and {2:,.0f} variables in {3:,.0f} minutes {4:.0f} seconds!'.format(onestate, acs5_df.shape[0], acs5_df.shape[1], np.floor((e-s)/60), (e-s) % 60))
        #print('\n')
        
    print('\n')


#            
#            this_seq_estimates_df = this_seq_estimates_df.append(estimates_onestate_df, sort=False)
#            this_seq_margins_of_error_df = this_seq_margins_of_error_df.append(margins_of_error_onestate_df, sort=False)


#    for this_state in states:
#        print('{0:}{1:.0f}'.format(this_state, thisyear))

#print('backing up...')
#metadata_df_bk = metadata_df

print('Grand total: {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor(g/60), g%60))
#metadata_df[varlist]

Parsing 2010...
Parsed 2010 metadata in 8 seconds!
ut: Wrote 2,278 geographies and 130 variables in 1 minutes 34 seconds!
va: Wrote 7,239 geographies and 130 variables in 3 minutes 50 seconds!
vt: Wrote 706 geographies and 130 variables in 0 minutes 54 seconds!
wa: Wrote 6,241 geographies and 130 variables in 3 minutes 21 seconds!
wi: Wrote 5,898 geographies and 130 variables in 3 minutes 25 seconds!
wv: Wrote 2,076 geographies and 130 variables in 1 minutes 43 seconds!
wy: Wrote 542 geographies and 130 variables in 0 minutes 53 seconds!


Grand total: 15 minutes 48 seconds!


# Write out metadata files for Baltimore

metafiles = [x for x in os.listdir(acs5_dir) if ('metadata' in x)]
for thismeta in metafiles:
    copyfile(acs5_dir+thismeta, baltimore_dir+'acs5_metadata/'+thismeta)
print('Done')


# Get Baltimore data

md_acs5_df = pandas.DataFrame()
datafiles = [x for x in os.listdir(acs5_dir) if (('tracts' in x) and ('md' in x))]
for thisfile in datafiles:
    xdf = pandas.read_csv(acs5_dir+thisfile, encoding='utf-8', low_memory=False)
    xdf = xdf.assign(year = int(thisfile[-10:-6]))
    md_acs5_df = md_acs5_df.append(xdf)
    
column_order = md_acs5_df.columns[0:1].tolist()
column_order += md_acs5_df.columns[-1:].tolist()
column_order += md_acs5_df.columns[1:-1].tolist()
#column_order += md_acs5_df.columns[-3:].tolist()

md_acs5_df[column_order][md_acs5_df['GEOID'].apply(lambda x: 'US24510' in x)].to_csv(baltimore_dir+'acs5_2010_2017.csv', encoding='utf-8')
print('Done!')


## SEQUENCE GUIDE FOR ACS 5-yr in 2017

Seq 1: unweighted count

Seq 2-3: age-sex

Seq 4: race

Seq 5: hispanic origin

Seq 6-7: ancestry

Seq 8-12: foreign birth

Seq 13-15: Place of Birth - Native

Seq 16-22: Residence Last Year - Migration

Seq 23-33: Journey to Work

Seq 34: Children - Relationship

Seq 35: Grand(Persons) - Age of HH Members

Seq 36-37: Households - Families

Seq 38-39: Marital Status

Seq 40: Fertility

Seq 41-42: School Enrollment

Seq 43-44: Educational Attainment

Seq 45-47: Language

Seq 48-56: Poverty

Seq 57-58: Disability

Seq 59-66: Income

Seq 67-72: Earnings

Seq 73-74: Veteran Status

Seq 75: Transfer Programs

Seq 76-79: Employment Status

Seq 80-102: Industry-Occupation-Class of Worker

Seq 103-112: Housing

Seq 113-123: Group Quarters

Seq 124-127: Health Insurance

Seq 128 Computer and Internet Usage

Seq 129: Quality Measures

Seq 130-133: Imputations

In [ ]:
#z = pandas.read_csv('/home/idies/workspace/Temporary/raddick/cra_scratch_final/acs5/acs5_tracts_for_cra_2017md.csv', encoding='utf-8', low_memory=False, index_col='GEOID')
#z



thisyear = 2017
s = time.time()

basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'
yeardir = basedir + str(thisyear) + '/'
if (thisyear == 2017):
    rawdatadir = yeardir + 'rawdata/'
    metadir = yeardir + 'metadata/xls_temp/'
elif (thisyear == 2016):
    rawdatadir = yeardir + 'rawdata/data/tab4/sumfile/prod/2012thru2016/group2/'
    metadir = yeardir + 'metadata/templates/'
elif (thisyear == 2015):
    rawdatadir = yeardir + 'rawdata/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2014):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2010thru2014/group2/'
    metadir = yeardir + 'metadata/seq/'
elif (thisyear == 2013):
    rawdatadir = yeardir + 'rawdata/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2012):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2008thru2012/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2011):
    rawdatadir = yeardir + 'rawdata/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2010):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2006thru2010/group2/'
    metadir = yeardir + 'metadata/'
    
estimates_dir = yeardir + 'estimates/'
margin_of_error_dir = yeardir + 'margin_of_error/'
vardir = yeardir + 'variables/'

geodir = yeardir + 'geography/'

extrasdir = '/home/idies/workspace/Storage/raddick/census/extras/'

outdir = acs5_dir
#outdir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/acs5/'

for thisdir in [estimates_dir, margin_of_error_dir, vardir, geodir]:#[datadir, errordir, vardir, geodir]:
    #print(thisdir)
    if not(os.path.exists(thisdir)):
        os.makedirs(thisdir)
e = time.time()
g = g + (e-s)

print('Done in {0:,.0f} seconds!'.format(e-s))
#os.chdir(rawdatadir)
#print(os.getcwd())